In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

In [2]:
X = np.linspace(-5, 5, 2000)
y = -2 * X + 5 + np.random.rand() * 0.75

In [3]:
# Problem definition
m = len(X)
def J(theta):
    h = theta[0] * X + theta[1] 
    error = h - y
    return 1 / (2 * m) * sum(error**2)

def predict(theta):
    h = theta[0] * X + theta[1] 
    return h

nVar = 2
VarMin = [-5,-10]
VarMax = [10,8]  
VarSize = np.array([nVar,1])
bounds = np.column_stack((VarMin,VarMax))

In [4]:
bounds

array([[ -5,  10],
       [-10,   8]])

In [5]:
class Particle_swarm:
      def __init__(self, J, bounds, VarSize):
        self.position = np.random.uniform(bounds[:, 0], bounds[:, 1]).reshape(VarSize)
        self.cost = J(self.position)
        self.velocity = np.zeros(VarSize)
        self.best_position = self.position.copy()
        self.best_cost = self.cost

In [6]:
class GlobalPractice:
      def __init__(self):
        self.cost = np.inf
        self.position = []

In [7]:
# Parameters of PSO
W = 0.9
c1 = 2
c2 = 1.5
nPop = 50
MaxIt = 10
GlobalBest = GlobalPractice()
particle = [None] * nPop

In [8]:
X_particle = [None] * nPop
y_particle = [None] * nPop

for i in range(nPop):
    particle[i] = Particle_swarm(J, bounds, VarSize)
    X_particle[i] = particle[i].position[0][0]
    y_particle[i] = particle[i].position[1][0]
    
    if particle[i].cost < GlobalBest.cost:
        GlobalBest.position = particle[i].position.copy()
        GlobalBest.cost = particle[i].cost
        X_GlobalBest = GlobalBest.position[0][0]
        y_GlobalBest = GlobalBest.position[1][0]

In [9]:
def Particle_Swarm_Optimization(particle, GlobalBest, cost_function, X_particle, y_particle, X_GlobalBest, y_GlobalBest):     
    plt.figure(figsize = (20,20))
    plt.subplot(1, 2, 1)
    plt.title('Particle Swarm Optimization', fontsize = 25)
    plt.xticks([])
    plt.yticks([])
    particles_plot = plt.scatter(X_particle, y_particle, marker = '*', label = 'Particles')
    GlobalBest_plot = plt.scatter(X_GlobalBest, y_GlobalBest, marker = '*', s = 100, color = 'r', label = 'GlobalBest')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.title('Regression line', fontsize = 25)
    plt.xticks([])
    plt.yticks([])
    y_pred = predict(GlobalBest.position)
    y_true = plt.plot(X, y, color = 'r', linewidth = 2, label = 'y_true')
    line = plt.plot(X, y_pred, color = 'b', linestyle = '--', linewidth = 2, label = 'y_predict')
    plt.legend()
    
    for iter in range(MaxIt):
        for i in range(nPop):
            # Update particles’ velocities.
            particle[i].velocity = W * particle[i].velocity +\
                                  np.random.rand() * c1 * (particle[i].best_position - particle[i].position) +\
                                  np.random.rand() * c2 * (GlobalBest.position - particle[i].position)                              

            # Move particles to their new positions.
            particle[i].position = particle[i].position + particle[i].velocity

            # if the position is out of boundries replace position with boundries
            particle[i].position = np.maximum(bounds[:, 0].reshape(-1, 1), np.minimum(bounds[:, 1].reshape(-1, 1), particle[i].position))

            # Update cost function
            particle[i].cost = cost_function(particle[i].position)

            # If a particle’s present position is better than its previous best position, update it.
            if particle[i].cost < particle[i].best_cost:
                particle[i].best_cost = particle[i].cost
                particle[i].best_position = particle[i].position.copy()

                # If a particle’s present position is better than Global best position, update it.
                if particle[i].cost < GlobalBest.cost:
                    GlobalBest.cost = particle[i].cost         
                    GlobalBest.position = particle[i].position.copy()

            # Modify the line parameters in a for loop
            X_particle[i] = particle[i].position[0][0]
            y_particle[i] = particle[i].position[1][0]

            X_GlobalBest = GlobalBest.position[0][0]
            y_GlobalBest = GlobalBest.position[1][0]

            # Update the plots
            plt.setp(particles_plot, offsets = np.column_stack((X_particle, y_particle)))
            plt.setp(GlobalBest_plot, offsets = np.column_stack((X_GlobalBest, y_GlobalBest)))

            y_pred = predict(GlobalBest.position)   

            plt.setp(line, linestyle='--', linewidth=2, ydata = y_pred)
            plt.pause(0.001)
        
    print(f'Best Cost is {GlobalBest.cost}')
    print(f'Best position is {GlobalBest.position}')

In [10]:
Particle_Swarm_Optimization(particle, GlobalBest, J, X_particle, y_particle, X_GlobalBest, y_GlobalBest)

Best Cost is 0.0055409632788617075
Best position is [[-2.03055862]
 [ 5.67332881]]
